In [43]:
from music21 import *
import glob
import pickle
import numpy as np
from keras.utils import np_utils

In [56]:
#Save all notes into string format
notes = []

for file in glob.glob("midi_songs2/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    if parts: 
        notes_to_parse = parts.parts[0].recurse()
    else: 
        notes_to_parse = midi.flat.notes
    
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

print(len(notes))

2125


In [3]:
with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

In [4]:
#Select sequence
sequence_length = 50

#Transform notes to integers
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

In [55]:
#Create input and output sequences
network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
print(n_patterns)

2075


In [31]:
# Reshape the input into a format compatible with LSTM layers and normalize
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input.shape

(2075, 50, 1)

In [54]:
# Normalize 
n_vocab = len(notes)
print(n_vocab)
network_input = network_input / n_vocab
print(network_input.shape)
network_output = np_utils.to_categorical(network_output)
print(network_output.shape)

2125
(2075, 50, 1)
(2075, 129, 2, 2, 2, 2, 2, 2, 2, 2, 2)


In [63]:
len(set(notes))

129

### Build Model

In [57]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

In [60]:
model = Sequential()

model.add(LSTM(
500,
return_sequences=True,
time_major=True
))
model.add(Dropout(0.3))
model.add(LSTM(500, return_sequences=True))
model.add(Dropout(0.3)) 
model.add(LSTM(500))
model.add(Dense(250))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [ ]:
model.fit(network_input, network_output, epochs=30, batch_size=64, callbacks=callbacks_list)

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
filepath,
monitor="loss",
verbose=0,
save_best_only=True,
mode="min"
)
callbacks_list = [checkpoint]